# Fine-tuning LLaMA 3.1 8B Instruct on Google Colab

This notebook installs required libraries, loads the base model, prepares data, and runs LoRA fine-tuning using **Unsloth** for efficiency.


In [ ]:
!pip install unsloth transformers accelerate bitsandbytes datasets trl

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = 'meta-llama/Llama-3.1-8b-instruct'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=['q_proj','k_proj','v_proj','o_proj'],
    lora_alpha=16,
    lora_dropout=0.05,
)

In [ ]:
from datasets import load_dataset

# Example dataset — user should replace with their own
dataset = load_dataset('json', data_files='train.json')

def format(example):
    return {
        'text': f"<s>[INST] {example['instruction']} [/INST] {example['output']} </s>"
    }

dataset = dataset.map(format)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='llama-3.1-8b-finetune',
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    warmup_steps=5,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset['train'],
    dataset_text_field='text',
    args=training_args,
)

trainer.train()

In [ ]:
model.save_pretrained('llama-finetuned')
tokenizer.save_pretrained('llama-finetuned')